In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [4]:
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    # Set TensorFlow to use only the GPU devices
    tf.config.experimental.set_visible_devices(gpus, 'GPU')
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)  
print(gpus)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
folder_path = r"F:\ML Python VsCode\Age_Gender_Emotion_Prediction_using_CNN\UTKFace"

In [6]:
age = []
gender = []
image = []
for file in os.listdir(folder_path):
    age.append(int(file.split('_')[0]))
    gender.append(int(file.split('_')[1]))
    image.append(file)

<style>
.output_area pre {
    color: white; /* Change output text color to white */
}
</style>

In [8]:
df = pd.DataFrame({'age':age,'gender':gender,'image':image})

In [9]:
df.sample(4)

,age,gender,image
11721,32,0,32_0_1_20170113141930308.jpg.chip.jpg
11922,32,1,32_1_0_20170103181859185.jpg.chip.jpg
21516,68,0,68_0_0_20170104213011828.jpg.chip.jpg
2480,1,1,1_1_2_20161219211512447.jpg.chip.jpg


In [11]:
df.shape

(23708, 3)

In [12]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [14]:
train_df.shape

(20000, 3)

In [15]:
test_df.shape

(3708, 3)

In [18]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range = 0.1,
    zoom_range = 0.05,
    horizontal_flip = True
)
test_datagen = ImageDataGenerator(
    rescale = 1./255
)

In [20]:
train_generator = train_datagen.flow_from_dataframe(train_df,directory=folder_path,x_col='image',y_col=['age','gender'],target_size=(200,200),class_mode='multi_output')
test_generator = test_datagen.flow_from_dataframe(test_df,directory=folder_path,x_col='image',y_col=['age','gender'],target_size=(200,200),class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


## Model Preperation

In [21]:
from keras.applications import VGG16
from keras.layers import *
from keras.models import Model

In [22]:
vggnet = VGG16(include_top = False,input_shape=(200,200,3))

In [23]:
vggnet.trainable = False
output = vggnet.layers[-1].output
flatten = Flatten()(output)

dense1 = Dense(512,activation='elu')(flatten)
dense2 = Dense(512,activation='elu')(flatten)

dense3 = Dense(512,activation='elu')(dense1)
dense4 = Dense(512,activation='elu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [24]:
model = Model(inputs=vggnet.input,outputs=[output1,output2])

In [30]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 200, 200, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 200, 200, 64  36928       ['block1_conv1[0][0]']           
                                )                                                             

In [34]:
model.compile(optimizer='adam',loss={'age':'mae','gender':'binary_crossentropy'},metrics={'age':'mae','gender':'accuracy'},loss_weights={'age':20,'gender':80})

In [ ]:
model.fit(train_generator,epochs=20,validation_data=test_generator)

Epoch 1/20
625/625 [==============================] - 621s 993ms/step - loss: 235.1607 - age_loss: 9.8062 - gender_loss: 0.4880 - age_mae: 9.8062 - gender_accuracy: 0.7630 - val_loss: 192.5024 - val_age_loss: 8.2396 - val_gender_loss: 0.3464 - val_age_mae: 8.2396 - val_gender_accuracy: 0.8484
Epoch 2/20
625/625 [==============================] - 181s 290ms/step - loss: 207.9047 - age_loss: 8.7391 - gender_loss: 0.4140 - age_mae: 8.7391 - gender_accuracy: 0.8058 - val_loss: 189.5595 - val_age_loss: 8.0457 - val_gender_loss: 0.3581 - val_age_mae: 8.0457 - val_gender_accuracy: 0.8484
Epoch 3/20
625/625 [==============================] - 184s 294ms/step - loss: 203.4116 - age_loss: 8.5491 - gender_loss: 0.4054 - age_mae: 8.5491 - gender_accuracy: 0.8083 - val_loss: 180.3287 - val_age_loss: 7.6933 - val_gender_loss: 0.3308 - val_age_mae: 7.6933 - val_gender_accuracy: 0.8568
Epoch 4/20
625/625 [==============================] - 183s 293ms/step - loss: 200.6972 - age_loss: 8.4767 - gender_los

In [ ]:
mode